In [71]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.nn as nn
import torch.optim as optim
import time
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'/Users/dennis/Documents/myGit/main/myUtils')

from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

In [72]:
from torch.utils.data import Dataset
from optModelRd import optModelRd

In [92]:
num_items = 2
num_data = 10
num_features = 5
alpha = 0.5
Q = 10
x = np.random.rand(num_data, num_features)
r = np.random.rand(num_data,num_items)
c = np.random.rand(num_data,num_items)

In [87]:
r.shape

(10, 2)

In [94]:
class optModelRd:
    def __init__(self, num_items, num_data, alpha, Q, r, c):
        self.num_items = num_items
        self.num_data = num_data
        self.alpha = alpha
        self.Q = Q
        self.r = r
        self.c = c
        self.d = cp.Variable(num_items)

        self.setObj(r, c)

    
    def setObj(self, r, c):
        """
        A method to set the objective function and constraints

        Args:
            r, c: Problem parameters
        """
        # TODO


    def solve(self):
        """
        Solve the optimization problem
        """
        self.problem.solve()
        opt_sol = self.d.value
        opt_val = self.problem.value
        return opt_sol, opt_val
    
    def solveC(self):
        """
        Solve the optimization problem using a closed-form solution
        """
        if self.alpha == 1:
            return print("Work in progress")
        else:
            S = np.sum(self.c ** (1 - 1 / self.alpha) * self.r ** (-1 + 1 / self.alpha))
            d_closed_form = (self.c ** (-1 / self.alpha) * self.r ** (-1 + 1 / self.alpha) * self.Q) / S
            optimal_value_closed_form = np.sum((self.r * d_closed_form) ** (1 - self.alpha)) / (1 - self.alpha)
        
        return d_closed_form, optimal_value_closed_form


In [95]:
optmodel = optModelRd(num_items, num_data, alpha, Q, r, c)

ValueError: Cannot broadcast dimensions  (10, 2) (2,)

In [77]:
optmodel.solve()

(array([2.29650983, 3.64826125, 4.05522891]), 9.931767208240155)

In [78]:
optmodel.solveC()

(array([2.29643147, 3.64841849, 4.05515004]), 9.93176721434811)

In [79]:
class optDatasetRd(Dataset):
    def __init__(self, x, r, c, Q, alpha):
        self.x = x
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data = r.shape[0]
        self.num_items = r.shape[1]
        self.num_features = x.shape[0]

        self.opt_sol = []
        self.opt_obj = []
        self.opt_sol_c = []
        self.opt_obj_c = []

    def solve(self):
        for i in range(self.num_data):
            optmodel = optModelRd(
                self.num_items, 
                self.num_data, 
                self.alpha, 
                self.Q, 
                self.r[i].reshape(-1), 
                self.c[i]
            )
            sol, val = optmodel.solve()

            self.opt_sol.append(sol)
            self.opt_obj.append(val)
        self.opt_sol = np.array(self.opt_sol)
        self.opt_obj = np.array(self.opt_obj)
    def solveC(self):
        for i in range(self.num_data):
            optmodel = optModelRd(
                self.num_items, 
                self.num_data, 
                self.alpha, 
                self.Q, 
                self.r[i].reshape(-1), 
                self.c[i]
            )
            sol, val = optmodel.solveC()
            self.opt_sol_c.append(sol)
            self.opt_obj_c.append(val)
        self.opt_sol_c = np.array(self.opt_sol_c)
        self.opt_obj_c = np.array(self.opt_obj_c)
    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.x[idx]),
            torch.FloatTensor(self.r[idx]),
            torch.FloatTensor(self.c[idx]),
            torch.FloatTensor(self.opt_sol[idx]),
            torch.FloatTensor([self.opt_obj[idx]]),
            torch.FloatTensor(self.opt_sol_c[idx]),
            torch.FloatTensor([self.opt_obj_c[idx]])
        )

In [80]:
dataset = optDatasetRd(x, r, c, Q, alpha)

IndexError: tuple index out of range